In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
df = pd.read_csv('CA_Honda_sub.csv', low_memory=False)
df.drop(columns = ['Unnamed: 0'],inplace = True)

In [3]:
df.columns

Index(['vin', 'price', 'msrp', 'miles', 'year', 'make', 'model', 'trim',
       'vehicle_type', 'body_type', 'body_subtype', 'drivetrain', 'fuel_type',
       'engine', 'engine_size', 'engine_measure', 'engine_aspiration',
       'transmission', 'speeds', 'doors', 'cylinders', 'interior_color',
       'exterior_color', 'base_exterior_color', 'base_interior_color',
       'scraped_at', 'status_date', 'first_scraped_at', 'dealer_id', 'street',
       'city', 'state', 'zip', 'latitude', 'longitude', 'seller_type',
       'inventory_type', 'car_address', 'car_street', 'car_city', 'car_state',
       'car_zip', 'dom', 'dom_180', 'dom_active', 'currency_indicator',
       'miles_indicator'],
      dtype='object')

In [4]:
index = []
for i in range(len(df)):
    if df['price'].iloc[i] < 50000:
        index.append(i)
df = df.iloc[index]

In [5]:
df_describe = df.describe()
df_describe.to_csv('describe_CA.csv')

In [6]:
df.dropna(subset=['price','trim','miles'],inplace = True)
print(len(df))

432136


In [7]:
print(len(set(df['vin'])))

51101


In [8]:
df.isnull().sum()

vin                         0
price                       0
msrp                   126427
miles                       0
year                        0
make                        0
model                       0
trim                        0
vehicle_type                0
body_type                   0
body_subtype           432136
drivetrain                  0
fuel_type                   0
engine                      0
engine_size                 0
engine_measure         432136
engine_aspiration      432136
transmission                0
speeds                 432136
doors                       0
cylinders                   0
interior_color         120623
exterior_color           6707
base_exterior_color    102146
base_interior_color    192689
scraped_at                  0
status_date                 0
first_scraped_at        90132
dealer_id                  15
street                   6816
city                      317
state                     316
zip                       316
latitude  

In [9]:
print(set(df['model']))

{'Civic', 'Accord', 'Fit'}


In [10]:
df.head()

,vin,price,msrp,miles,year,make,model,trim,vehicle_type,body_type,...,car_address,car_street,car_city,car_state,car_zip,dom,dom_180,dom_active,currency_indicator,miles_indicator
1,1HGCR2F38GA024262,15980.0,15980.0,52239.0,2016.0,Honda,Accord,LX,Car,Sedan,...,NaN,4355 Rosewood Drive,Pleasanton,CA,94588,10.0,5.0,5.0,NaN,NaN
2,2HGFC2F76HH538521,21000.0,NaN,36063.0,2017.0,Honda,Civic,EX,Car,Sedan,...,NaN,11151 Laurel Canyon Blvd.,Mission Hills,CA,91340,45.0,40.0,40.0,NaN,NaN
3,2HGFC2F78GH537210,16790.0,16790.0,45526.0,2016.0,Honda,Civic,EX,Car,Sedan,...,NaN,580 Auto Park Dr,Chula Vista,CA,91911,32.0,15.0,15.0,NaN,NaN
4,JHMGK5H54GX021665,13832.0,13832.0,34087.0,2016.0,Honda,Fit,LX,Car,Hatchback,...,NaN,4590 Stevens Creek Blvd,San Jose,CA,95129,13.0,12.0,12.0,NaN,NaN
5,1HGCR2F36HA225028,17994.0,17994.0,29764.0,2017.0,Honda,Accord,LX,Car,Sedan,...,NaN,1720 Santa Monica Blvd,Santa Monica,CA,90404,81.0,60.0,60.0,NaN,NaN


In [11]:
df_vin = df.groupby(['vin','miles','model','trim']).mean()
df_vin.drop(columns = ['body_subtype', 'engine_measure', 'engine_aspiration', 'speeds', 'dom', 'dom_180', 'dom_active', \
                            'currency_indicator', 'miles_indicator'],inplace = True)
df_vin = df_vin.reset_index()

In [12]:
df_vin.head()

,vin,miles,model,trim,price,msrp,year,engine_size,doors,cylinders,scraped_at,status_date,first_scraped_at,dealer_id,latitude,longitude
0,19XFA15339E036897,156277.0,Civic,VP,4865.000000,NaN,2009.0,1.8,4.0,4.0,1.500851e+09,1.501724e+09,1.497852e+09,1.079409e+06,37.390000,-121.900000
1,19XFA15559E019326,98000.0,Civic,LX,9998.000000,NaN,2009.0,1.8,4.0,4.0,1.561755e+09,1.562094e+09,1.561183e+09,1.053075e+06,35.293000,-119.028100
2,19XFA16319E014945,51075.0,Civic,VP,8491.000000,8491.000000,2009.0,1.8,4.0,4.0,1.482616e+09,1.482718e+09,NaN,1.008432e+06,37.551818,-115.852727
3,19XFA16319E020986,110589.0,Civic,VP,9114.214286,9114.214286,2009.0,1.8,4.0,4.0,1.482732e+09,1.484386e+09,1.481267e+09,1.022845e+06,34.550000,-85.305714
4,19XFA16389E027188,168321.0,Civic,VP,5991.000000,5991.000000,2009.0,1.8,4.0,4.0,1.487055e+09,1.487065e+09,1.487056e+09,1.001170e+06,37.307500,-116.916250


In [13]:
len(df_vin)

56663

In [15]:
set(df_vin['trim'])

{'3.0 EX',
 '3.0 LX',
 '3.0 LX-SE',
 '3.0 SE',
 'B1',
 'Base',
 'CX',
 'DX',
 'EX',
 'EX LEATHER',
 'EX Leather',
 'EX V6',
 'EX W/LEATHER',
 'EX-L',
 'EX-L V-6',
 'EX-L V6',
 'EX-T',
 'GX',
 'HF',
 'HX',
 'HX CVT',
 'HYBRID',
 'Hybrid',
 'Hybrid CVT',
 'Hybrid CVT PZEV',
 'Hybrid EX',
 'Hybrid EX-L',
 'Hybrid Touring',
 'LX',
 'LX-P',
 'LX-S',
 'LX-SE',
 'NGV',
 'Natural Gas',
 'Plug-In',
 'SE',
 'SI',
 'SPECIAL EDITION',
 'Si',
 'Sport',
 'Sport SE',
 'Touring',
 'Touring V6',
 'VALUE PACKAGE',
 'VP'}